<a href="https://colab.research.google.com/github/maverick98/CDS/blob/main/DNN_Assignment2a_Template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Group No 189

## Group Member Names:
• MAHESH NARAM (2023aa05876): 100%

• GIRIJA SHANKAR SAHOO (023AA05235): 100%

• SOURAJEET SAHOO (023aa05029): 100%

• MANO RANJAN SAHU (2023aa05738): 100%
## Journal used for the implementation
Journal title: Wav2vec 2.0: A Framework for Self-Supervised Learning of Speech Representations  
Authors: Alexei Baevski, Henry Zhou, Abdelrahman Mohamed, Michael Auli  
Journal Name: NeurIPS  
Year: 2020


In [1]:
!pip install -qq jiwer
!pip install -qq datasets


# 1. Import the required libraries

In [6]:
##---------Type the code below this line------------------##
import torch
import torch.nn as nn
from transformers import Wav2Vec2Model, Wav2Vec2Processor
import librosa
import numpy as np
import os
import tarfile
import requests
from jiwer import wer
import evaluate



# 2. Data Acquisition

For the problem identified by you, students have to find the data source themselves from any data source.

Provide the URL of the data used.

Write Code for converting the above downloaded data into a form suitable for DL



In [7]:
##---------Type the code below this line------------------##


url = "https://www.openslr.org/resources/12/dev-clean.tar.gz"
tar_path = "dev-clean.tar.gz"
audio_dir = "LibriSpeech/dev-clean"

# Download the tar.gz file
if not os.path.exists(tar_path):
    print("Downloading dataset...")
    response = requests.get(url)
    with open(tar_path, "wb") as f:
        f.write(response.content)

# Extract the tar.gz file
if not os.path.exists(audio_dir):
    print("Extracting dataset...")
    with tarfile.open(tar_path, "r:gz") as tar:
        tar.extractall()



# 3. Data Preparation

Perform the data prepracessing that is required for the data that you have downloaded.


This stage depends on the dataset that is used.

In [8]:
##---------Type the code below this line------------------##

# Let's list all audio files in the dataset directory (dev-clean)
audio_files = []
for root, dirs, files in os.walk(audio_dir):
    for file in files:
        if file.endswith(".flac"):
            audio_files.append(os.path.join(root, file))

print(f"Found {len(audio_files)} audio files.")

# For this example, we'll use only one file to demonstrate the workflow
audio_file = audio_files[0]

# Load audio using librosa at 16kHz (Wav2Vec2 model works at 16kHz)
data, sampling_rate = librosa.load(audio_file, sr=16000)

# Step 4: Preprocess audio data using Wav2Vec2 processor
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base")
input_values = processor(data, return_tensors="pt", sampling_rate=16000).input_values

# Splitting into train/test sets for demonstration purposes
train_data, test_data = input_values[:, :int(0.8*input_values.size(-1))], input_values[:, int(0.8*input_values.size(-1)):]



Found 2703 audio files.


/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:364: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


## 4. Deep Neural Network Architecture


## 4.1 Design the architecture that you will be using

* CNN / RNN / Transformer as per the journal referenced



In [9]:
##---------Type the code below this line------------------##
class Wav2Vec2ForSpeechRecognition(nn.Module):
    def __init__(self):
        super(Wav2Vec2ForSpeechRecognition, self).__init__()
        # Load Wav2Vec2 model for feature extraction
        self.wav2vec = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base")
        # Linear layer for final classification (assuming character-level classification)
        self.classifier = nn.Linear(self.wav2vec.config.hidden_size, 29)  # Adjust 29 if needed

    def forward(self, input_values):
        # Extract features from Wav2Vec2
        features = self.wav2vec(input_values).last_hidden_state
        # Pass through classifier
        logits = self.classifier(features)
        return logits



## 4.2 DNN Report

Report the following and provide justification for the same.

* Number of layers
* Number of units in each layer
* Total number of trainable parameters



# 5. Training the model


In [13]:
# Configure the training, by using appropriate optimizers, regularizations and loss functions
##---------Type the code below this line------------------##
# Model initialization
model = Wav2Vec2ForSpeechRecognition()

# Optimizer and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=5e-5)
loss_function = nn.CTCLoss()  # CTC loss for speech recognition

# Simplified training loop (you can expand this for more epochs and larger data)
epochs = 10  # Set epochs as needed
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()

    logits = model(train_data)

    # Dummy target for demo purposes (replace with real transcriptions)
    target = torch.randint(0, 29, (train_data.size(0), int(0.5*logits.size(1))), dtype=torch.long)  # Random dummy target
    input_lengths = torch.full((train_data.size(0),), logits.size(1), dtype=torch.long)
    target_lengths = torch.full((train_data.size(0),), target.size(1), dtype=torch.long)

    # Calculate CTC Loss
    loss = loss_function(logits.permute(1, 0, 2), target, input_lengths, target_lengths)
    loss.backward()
    optimizer.step()

    print(f"Epoch {epoch + 1}, Loss: {loss.item()}")

Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1, Loss: -1.7351213693618774
Epoch 2, Loss: -1.6524814367294312
Epoch 3, Loss: -1.4827373027801514
Epoch 4, Loss: -1.4014805555343628
Epoch 5, Loss: -1.223551630973816
Epoch 6, Loss: -1.020250678062439
Epoch 7, Loss: -0.8831086158752441
Epoch 8, Loss: -0.7134530544281006
Epoch 9, Loss: -0.5902512073516846
Epoch 10, Loss: -0.4782578647136688


# 6. Test the model


In [16]:
##---------Type the code below this line------------------##
model.eval()
with torch.no_grad():
    logits = model(test_data)
    predictions = torch.argmax(logits, dim=-1)
    print(f"Shape of predictions: {predictions}")
    print(f"Shape of target: {target}")


Shape of predictions: tensor([[15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15,
         15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15,
         15, 15, 15, 15]])
Shape of target: tensor([[20,  6, 26, 13,  4, 12, 18, 17,  5, 12,  3, 16, 21, 28, 21,  5, 11, 26,
          2, 15,  4, 22, 26, 27, 27, 13,  9,  2, 14,  7,  5, 11, 28, 18, 13, 16,
         19,  6, 21,  6,  3, 12,  7, 14, 17, 13, 28, 11, 14,  9, 19, 16, 10, 23,
          7,  7, 26,  0, 28, 10,  0,  4,  8, 25, 10, 27, 19, 12, 11, 25,  1,  5,
         12, 20,  4, 28, 12, 24, 10, 22,  7]])


# 7. Report the result

1. Plot the training and validation accuracy history.
2. Plot the training and validation loss history.
3. Report the testing accuracy and loss.
4. Show Confusion Matrix for testing dataset.
5. Report values for preformance study metrics like accuracy, precision, recall, F1 Score.


In [15]:
##---------Type the code below this line------------------##
def greedy_decode(logits, processor):
    """Greedy decoding for converting logits to text."""
    pred_ids = torch.argmax(logits, dim=-1)
    pred_transcriptions = processor.batch_decode(pred_ids)
    return pred_transcriptions

# Decode the predictions
decoded_preds = greedy_decode(logits, processor)

# For demonstration purposes, we'll assume dummy targets (replace these with real transcriptions from the dataset)
decoded_target = processor.batch_decode(target)

# Step 9: Calculate Word Error Rate (WER)
wer_score = wer(decoded_target, decoded_preds)
print(f"Word Error Rate (WER): {wer_score * 100:.2f}%")


Word Error Rate (WER): 100.00%


### NOTE


All Late Submissions will incur a <b>penalty of -2 marks </b>. So submit your assignments on time.

Good Luck